In [3]:
import numpy as np
import scipy as sp
import pandas as pd
import util

In [4]:
test_df = pd.read_csv('data/2010_2019_daily_data.csv')
target = np.array([.4405, .2694, .1375, .1526])
test_df = test_df[['SPX Index', 'SHCOMP Index', 'SENSEX Index', 'MXLA Index']].to_numpy()
test_df = test_df[1828:]
Dates = pd.read_csv('data/2010_2019_daily_data.csv')['Dates']
start_day = Dates[1828]
end_day = Dates[len(Dates)-1]

daily_growth = util.daily_growth(test_df)


In [10]:
mu=np.mean(daily_growth, axis=0)*252
cov=np.cov(daily_growth.T)*252
alpha = 1
def utility(w):
    return w @ mu - alpha / 2 * w.T@cov@w

In [12]:
utility(np.array([.2,.3,.1,.4]))

0.058592046964779765

In [25]:
x0 = np.ones(4) / 4
cons = ({'type': 'eq', 'fun': lambda x: x.sum() - 1.0})
bnds = [(0, 1)] * 4
w = sp.optimize.minimize(utility, x0, method='SLSQP', bounds=bnds, constraints=cons)['x']

In [26]:
w

array([8.88178420e-16, 1.00000000e+00, 2.47244211e-16, 0.00000000e+00])

In [27]:
cov


array([[0.01653424, 0.00287652, 0.00214251, 0.01042179],
       [0.00287652, 0.02552402, 0.00367652, 0.00401759],
       [0.00214251, 0.00367652, 0.01382135, 0.00276823],
       [0.01042179, 0.00401759, 0.00276823, 0.03989487]])